<a href="https://colab.research.google.com/github/samuel0711/image-anomaly-detection/blob/main/isolation_tcc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import io
import sys
import os
import gc
import shutil
import numpy as np
import matplotlib.pyplot as plt
from skimage import io as skio
import torch
from torch import nn #neural networks
import torch.nn.functional as F
from torchvision.utils import save_image


import time

#Carregamento de Dados
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from sklearn.ensemble import IsolationForest
from sklearn.metrics import accuracy_score


if 'google.colab' in sys.modules:
    from google.colab import drive
    drive.mount('/content/drive')
    os.chdir("/content/drive/My Drive/Colab Notebooks")
    sys.path.append('/content/drive/My Drive/Colab Notebooks')

Mounted at /content/drive


In [3]:
args = {
    'epoch':100,          #Numero de épocas
    'lr':5e-4,            #Taxa de aprendizado
    'weight_decay':5e-5,  #Penalidade L2 (Regularização)
    'batch_size': 50,     #Tamanho do batch
    'num_workers': 2
}

if torch.cuda.is_available():
  args['device'] = torch.device('cuda')
else:
  args['device'] = torch.device('cpu')

print(args['device'])


train_set = np.load('train_dataset.npy', allow_pickle = True)/255.0
test_set = np.load('test_dataset.npy', allow_pickle = True)/255.0

cuda


In [4]:
print('Treino\nMédia: ',train_set.mean(),'\t Desvio Padrão: ',train_set.std(),'\n\n')
print('Teste\nMédia: ',test_set.mean(),'\t Desvio Padrão: ',test_set.std())

/usr/local/lib/python3.7/dist-packages/numpy/core/_methods.py:212: RuntimeWarning: overflow encountered in reduce
  arrmean = umr_sum(arr, axis, dtype, keepdims=True, where=where)


Treino
Média:  0.3945 	 Desvio Padrão:  inf 


Teste
Média:  0.44 	 Desvio Padrão:  inf


In [5]:
train_set = torch.from_numpy(train_set)
test_set = torch.from_numpy(test_set)

In [6]:
train_loader = DataLoader(train_set, 
                          batch_size=args['batch_size'], 
                          shuffle=True, 
                          num_workers=args['num_workers'])

test_loader = DataLoader(test_set, 
                         batch_size=args['batch_size'], 
                         shuffle=True, 
                         num_workers=args['num_workers']) 

In [7]:
#Define the Convolutional Autoencoder
class ConvAutoencoder(nn.Module):
    def __init__(self):
        super(ConvAutoencoder, self).__init__()
       
        #Encoder
        self.conv1 = nn.Conv2d(1, 64, 3, padding = 1)  #in_channel, out_channel, kernel_size
        self.conv2 = nn.Conv2d(64, 128, 3, padding = 1)
        self.conv3 = nn.Conv2d(128, 128, 3, padding = 1) 
        self.conv4 = nn.Conv2d(128, 254, 3, padding = 1) 
        self.pool = nn.MaxPool2d(2) #kernel_size, stride
       
        #Decoder
        self.t_conv1 = nn.ConvTranspose2d(254, 128, 2, stride=2)        
        self.t_conv2 = nn.ConvTranspose2d(128, 128, 2, stride=2)
        self.t_conv3 = nn.ConvTranspose2d(128, 64, 2, stride=2)
        self.t_conv4 = nn.ConvTranspose2d(64, 1, 2, stride=2)


    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        x = F.leaky_relu(self.conv4(x))
        x = self.pool(x)
        x = F.relu(self.t_conv1(x))
        x = F.relu(self.t_conv2(x))
        x = F.relu(self.t_conv3(x))
        x = torch.sigmoid(self.t_conv4(x))
              
        return x


#Instantiate the model
model = ConvAutoencoder()


# **Treinando**

In [8]:
from torch import optim

criterio = nn.MSELoss().to(args['device'])
optimizer = optim.Adam(model.parameters(), lr=args['lr'])
model.to(args['device'])

ConvAutoencoder(
  (conv1): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(128, 254, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (t_conv1): ConvTranspose2d(254, 128, kernel_size=(2, 2), stride=(2, 2))
  (t_conv2): ConvTranspose2d(128, 128, kernel_size=(2, 2), stride=(2, 2))
  (t_conv3): ConvTranspose2d(128, 64, kernel_size=(2, 2), stride=(2, 2))
  (t_conv4): ConvTranspose2d(64, 1, kernel_size=(2, 2), stride=(2, 2))
)

In [9]:
def save_decod_img(img, output, epoch, name):
    img = img.view(img.size(0), 1, img.shape[2], img.shape[3])
    output = output.view(output.size(0), 1, output.shape[2], output.shape[3])
    error = output.view(output.size(0), 1, output.shape[2], output.shape[3]) - img.view(img.size(0), 1, img.shape[2], img.shape[3])
    save_image(img, './imgs_tcc/{}_imagem_image{}.png'.format(name,epoch))
    save_image(output, './imgs_tcc/{}_output_image{}.png'.format(name,epoch))
    save_image(error, './imgs_tcc/{}_error_image{}.png'.format(name,epoch))

## Treino ( Somente com imagens não-anomalas)

In [10]:
def train(train_loader, model, epoch, name):
  ###################
  # treinando o modelo #
  ###################
  model.train()
  
  start = time.time()

  tns = []

  train_loss = 0.0
  for data in train_loader:
    imagem = data.unsqueeze(1).float()
    imagem = imagem.to(args['device'])

    #imagem = imagem/255.0

    # clear the gradients of all optimized variables
    optimizer.zero_grad()

    # forward pass
    output = model(imagem)

    # calculando loss
    loss = criterio(output, imagem)
    #isolation_forest.fit(torch.flatten(output,start_dim=1).cpu().data)


    # backpropagation
    loss.backward()

    # próximo passo
    optimizer.step()

    # update running training loss
    train_loss += loss.item()

    if epoch == args['epoch']:
      for x, y in zip(imagem, output):
          tns.append(np.array(y.squeeze().cpu().data - x.squeeze().cpu().data))

  if epoch % 5 == 0:
    save_decod_img(imagem.cpu().data, output.cpu().data, epoch, name)


  train_loss = train_loss/len(train_loader)

  end = time.time()


  print('########## Train ##########')

  print("Epoca: {}\t Train Loss: {:.6f}, Time: {:.2f}\n".format(epoch, train_loss, end-start))
  #print("Medía do Loss:", epoch_loss_train.mean())

  return train_loss

## Validação ( Somente com imagens anômalas )

In [11]:
def validate(test_loader, model, epoch, name):
  ###################
  # validando o modelo #
  ###################
  model.eval()

  tns = []
  predicts = []
  outliers, inliers = 0, 0
  
  start = time.time()
  
  test_loss = 0.0
  with torch.no_grad():
    for data in test_loader:
      imagem = data.unsqueeze(1).float()
      imagem = imagem.to(args['device'])

      #imagem = imagem/255.0
      # forward pass
      output = model(imagem)

      #y = isolation_forest.predict(torch.flatten(output,start_dim=1).cpu().data)
      #outliers += list(y).count(-1)
      #inliers += list(y).count(1)
      #predicts.append(list(y).count(-1)/y.shape[0])

      # calculando loss
      loss = criterio(output, imagem)

      # update running test loss
      test_loss += loss.item()

      if epoch == (args['epoch']-1):
        for x, y in zip(imagem, output):
          tns.append(y.squeeze().cpu().numpy() - x.squeeze().cpu().numpy())

    test_loss = test_loss/len(test_loader)
    end = time.time()

    if epoch % 5 == 0:
      save_decod_img(imagem.cpu().data, output.cpu().data, epoch, name)
    print('########## Validate ##########')

    
    print("Epoca: {}\t Val Loss: {:.6f}, Time: {:.2f}\n".format(epoch, test_loss, end-start)) #\nAcurácia do Isolation Forest: {:.3f}\t {} Outliers, {} Inliers\n".format(epoch, test_loss, end-start, (sum(predicts)/len(predicts)), outliers, inliers))

    return test_loss#, (sum(predicts)/len(predicts))

## Rodando Validação e Treino


In [12]:
train_losses, val_losses = [], []
val_predicts = []

i = 1
for epoch in range(args['epoch']):
  #Train
  train_loss = train(train_loader, model, epoch, 'train')
  train_losses.append(train_loss)

  val_loss = validate(test_loader, model, epoch, 'test')
  val_losses.append(val_loss)
  #val_predicts.append(predicts)

########## Train ##########
Epoca: 0	 Train Loss: 0.012458, Time: 20.58

########## Validate ##########
Epoca: 0	 Val Loss: 0.003931, Time: 1.97

########## Train ##########
Epoca: 1	 Train Loss: 0.003347, Time: 17.83

########## Validate ##########
Epoca: 1	 Val Loss: 0.003025, Time: 1.91

########## Train ##########
Epoca: 2	 Train Loss: 0.002424, Time: 17.96

########## Validate ##########
Epoca: 2	 Val Loss: 0.002513, Time: 1.92

########## Train ##########
Epoca: 3	 Train Loss: 0.001959, Time: 18.09

########## Validate ##########
Epoca: 3	 Val Loss: 0.002266, Time: 1.92

########## Train ##########
Epoca: 4	 Train Loss: 0.001657, Time: 18.24

########## Validate ##########
Epoca: 4	 Val Loss: 0.002040, Time: 1.92

########## Train ##########
Epoca: 5	 Train Loss: 0.001463, Time: 21.22

########## Validate ##########
Epoca: 5	 Val Loss: 0.001930, Time: 1.98

########## Train ##########
Epoca: 6	 Train Loss: 0.001332, Time: 18.52

########## Validate ##########
Epoca: 6	 Val Loss: 

## Teste

In [13]:
train_set = None
test_set = None

In [14]:
'''isolation_forest = IsolationForest()

isolation_forest.fit( np.reshape(train_set, (train_set.shape[0],train_set.shape[1]*train_set.shape[2])) )

inlier_pred_test = isolation_forest.predict(np.reshape(train_set, (train_set.shape[0],train_set.shape[1]*train_set.shape[2])))
outlier_pred = isolation_forest.predict(np.reshape(test_set, (test_set.shape[0],test_set.shape[1]*test_set.shape[2])))

print("Accuracy in Detecting Legit Cases:", list(inlier_pred_test).count(1)/inlier_pred_test.shape[0])
print("Accuracy in Detecting Fraud Cases:", list(outlier_pred).count(-1)/outlier_pred.shape[0])'''

'isolation_forest = IsolationForest()\n\nisolation_forest.fit( np.reshape(train_set, (train_set.shape[0],train_set.shape[1]*train_set.shape[2])) )\n\ninlier_pred_test = isolation_forest.predict(np.reshape(train_set, (train_set.shape[0],train_set.shape[1]*train_set.shape[2])))\noutlier_pred = isolation_forest.predict(np.reshape(test_set, (test_set.shape[0],test_set.shape[1]*test_set.shape[2])))\n\nprint("Accuracy in Detecting Legit Cases:", list(inlier_pred_test).count(1)/inlier_pred_test.shape[0])\nprint("Accuracy in Detecting Fraud Cases:", list(outlier_pred).count(-1)/outlier_pred.shape[0])'

In [15]:
model.eval()

tns = []

for imagem in train_loader:
  imagem = imagem.unsqueeze(1).float()
  imagem = imagem.to(args['device'])

  # forward pass
  output = model(imagem)

  # calculando loss
  loss = criterio(output, imagem)

  for img in output:
    tns.append(img.squeeze().cpu().data.numpy())


print("Loss: ",loss.item())

Loss:  0.00044387156958691776


In [16]:
isolation_forest = IsolationForest()

inlier_dataset = np.array(tns)

inlier_dataset = np.reshape(inlier_dataset, (inlier_dataset.shape[0],inlier_dataset.shape[1]*inlier_dataset.shape[2]))

isolation_forest.fit(inlier_dataset)

IsolationForest()

In [17]:
model.eval()

tns = []

test_loss = 0.0
with torch.no_grad():
  for imagem in test_loader:
    imagem = imagem.unsqueeze(1).float()
    imagem = imagem.to(args['device'])

    # forward pass
    output = model(imagem)

    for img in output:
      tns.append(img.squeeze().cpu().data.numpy())


In [18]:
inlier_dataset.shape

(8000, 25600)

In [19]:
outlier_dataset = np.array(tns)

inlier_pred_test = isolation_forest.predict(inlier_dataset)
outlier_pred = isolation_forest.predict(np.reshape(outlier_dataset, (outlier_dataset.shape[0],outlier_dataset.shape[1]*outlier_dataset.shape[2])))

print("Accuracy in Detecting Legit Cases:", list(inlier_pred_test).count(1)/inlier_pred_test.shape[0])
print("Accuracy in Detecting Fraud Cases:", list(outlier_pred).count(-1)/outlier_pred.shape[0])

Accuracy in Detecting Legit Cases: 0.925625
Accuracy in Detecting Fraud Cases: 0.716


In [21]:
outlier_dataset.shape

(2000, 160, 160)

In [22]:
dataset = np.concatenate((inlier_dataset,np.reshape(outlier_dataset, (outlier_dataset.shape[0],outlier_dataset.shape[1]*outlier_dataset.shape[2]))))

label = np.concatenate((inlier_pred_test,outlier_pred))

dataset_pred = isolation_forest.predict(dataset)

print("Accuracy of all dataset: ",accuracy_score(label,dataset_pred))

Accuracy of all dataset:  1.0


## Plot Treino x Teste

In [ ]:
 plt.plot(train_losses, label='Train')
 plt.plot(val_losses, label='Test')
 plt.legend(loc="upper right")
 plt.ylabel('Loss')
 plt.xlabel('Epochs')

## Plot da Imagem Original(normalizada) x Imagem Reconstruída via AutoEncoder Convolucional

In [ ]:
# obtain one batch of test images
dataiter = iter(train_loader)
images = dataiter.next()

imagem = images.unsqueeze(1).float()
imagem /= 255.0
imagem = imagem.to(args['device'])

# get sample outputs
output = model(imagem)
# prep images for display
images = images.numpy()



# output is resized into a batch of iages
output = output.view(args['batch_size'], 1, 160, 160)
# use detach when it's an output that requires_grad
output = output.detach()#.numpy()



# plot the first ten input images and then reconstructed images
fig, axes = plt.subplots(nrows=2, ncols=10, sharex=True, sharey=True, figsize=(24,4))
for idx in np.arange(20):
    ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
    loss = criterio(output[idx], imagem[idx])
    label = 'Loss value: {:.3f}'
    ax.set_xlabel(label.format(loss) )
    ax.set_title("Reconstructed Image")
    plt.imshow(output[idx].squeeze().cpu(), cmap='gray')
    
# plot the first ten input images and then reconstructed images
fig, axes = plt.subplots(nrows=2, ncols=10, sharex=True, sharey=True, figsize=(24,4))
for idx in np.arange(20):
    ax = fig.add_subplot(2, 20/2, idx+1, xticks=[], yticks=[])
    loss = criterio(output[idx], imagem[idx])
    label = 'Loss value: {:.3f}'
    ax.set_xlabel(label.format(loss) )
    ax.set_title("Original Image")
    plt.imshow(images[idx].squeeze(), cmap='gray')
